In [1]:
# 딥러닝은 일반 기술이 아닌 학문이라고 한다.

In [6]:
# 곱셈 계층 구현

class MulLayer:

  # 초기화
  # 곱셈 계층으로 들어오고 있는 값을 저장할 변수 만들기
  # 저장하는 이유? backward 할 때 각각 반대로 곱해서 리턴해야 하기 때문에

  # -> x 방향으로 갈 때는 dout * y
  # -> y 방향으로 갈 때는 dout * x
  def __init__(self):
    self.x = None
    self.y = None

  # 노드로 들어온 값에 대한 곱을 수행한다.
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    
    return out

  # 다음 계층에서 역전파된 미분값(dout)에 흘러들어온 x, y 를 각각 바꿔서 곱한 값을 리턴
  def backward(self, dout):
    # 무조건 미분값이 리턴된다.
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

In [13]:
# 순전파를 이용해 사과 가격 확인하기
apple_price = 100
apple_count = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 먼저 사과 가격 곱하기 사과 개수의 결과를 구할 것
apple_price_total = mul_apple_layer.forward(apple_price, apple_count)
final_apple_price = mul_tax_layer.forward(apple_price_total, tax)

final_apple_price

220.00000000000003

In [14]:
# 역전파 수행하기
dprice = 1 # 최종 가격에 대한 미분. d가격 / d가격 = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_cnt  = mul_apple_layer.backward(dapple_price)

print("사과 가격에 대한 미분값 : {}".format(dapple))
print("사과 개수에 대한 미분값 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 : {}".format(dtax))

사과 가격에 대한 미분값 : 2.2
사과 개수에 대한 미분값 : 110.00000000000001
소비세에 대한 미분값 : 200


In [20]:
class AddLayer:
  # 덧셈 계층의 특징
  # 역전파 때 미분값이 들어오면 그대로 입력 신호에 거꾸로 흘려보내준다.

  def __init__(self):
    # 초기화 과정에서 변수로 입력된 값을 저장할 필요는 없다.
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    # 곱셈 계층의 코드 모양과 비슷하게 해주기 위해서 1을 곱했음
    dx = dout * 1
    dy = dout * 1

    return dx, dy

In [21]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층 만들기 각 과일의 개수 * 가격
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()

# 각 과일의 가격들을 더해주는 레이어
add_apple_orange_layer = AddLayer()

# 소비세를 적용시키는 레이어
mul_tax_layer = MulLayer()

# 순전파 수행
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
total_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(total_price, tax)

# 역전파 수행하기
dprice = 1
dtotal_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num   = mul_apple_layer.backward(dapple_price)

print("사과 2개 오렌지 3개의 가격 : {}".format(price))
print("사과 개수 미분 : {}".format(dapple_num))
print("사과 가격 미분 : {}".format(dapple))
print("오렌지 개수 미분 : {}".format(dorange_num))
print("오렌지 가격 미분 : {}".format(dorange))
print("소비세 미분 : {}".format(dtax))

사과 2개 오렌지 3개의 가격 : 715.0000000000001
사과 개수 미분 : 110.00000000000001
사과 가격 미분 : 2.2
오렌지 개수 미분 : 165.0
오렌지 가격 미분 : 3.3000000000000003
소비세 미분 : 650


## 신경망을 위한 각종 계층 구현하기
1. 활성화 함수 계층 구현하기
 * ReLU
 * Sigmoid

In [27]:
# ReLU
# 음수는 0으로 통일, 양수는 양수 그대로 사용
import numpy as np

x = np.arange(-5, 5)
mask = x <= 0
mask

x_relu = x.copy()
x_relu[mask] = 0
x_relu

array([0, 0, 0, 0, 0, 0, 1, 2, 3, 4])

In [28]:
class RuLU:
  
  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x <= 0) # 매개변수로 들어온 넘파이배열 x의 원소가 0 이하인지 판단하기
    out = x.copy() # 원본 배열 복사 후
    out[self.mask] = 0 # 0보다 작은 원소들을 0으로 만들기

    return out

  # backward 로 들어오는 미분값이 들어있는 배열은
  # forward 때와는 다르게 원래 0 이었던 곳에 0 이 아닌 값이 들어갈 수도 있다.
  # 따라서 원래 0이었던 곳 ( mask ) 은 전부 0 으로 다시 바꿔준다

  def backward(self, dout):

    # 미분값들이 들어있는 배열에다가도 마스킹 처리를 한다.
    dout[self.mask] = 0 # 상류에서 들어온 값에서 0보다 작은 값들에 대해 0으로 치환
    dx = dout # 완성된 ReLU 배열 리턴

    return dx

In [33]:
# 시그모이드 계층 만들기
# 시그모이드 함수가 y라면, 시그모이드 함수를 미분했을 때는 y(1-y) 이다.
# 즉, 순전파 진행 시에 y의 결과값을 저장하고 있으면, 이 값을 그대로 역전파때 활용할 수 있다는 이야기

class Sigmoid:
  # 순전파 할때 계산되었던 값을 가지고 있는다.
  def __init__(self):
       self.out = None


  def forward(self, x):
        out = 1 / ( 1 + np.exp(-x))
        self.out = out
        
        return out
    
  def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

In [34]:
# 전치 행렬
X = np.arange(6).reshape(2, 3)
X

array([[0, 1, 2],
       [3, 4, 5]])

In [35]:
X.T

array([[0, 3],
       [1, 4],
       [2, 5]])

In [41]:
class Affine:
  # 초기 가중치, 초기편향 가지고 있기
  def __init__(self, W, b):
    self.W = W
    self.b = b

    self.X = None
    self.original_x_shape = None # 원본 데이터의 모양을 유지하기 위한 변수
    
    # 경사하강법을 통해 가중치를 업데이트 해야 해서, 기존 미분값을 알고 있어야 한다.
    # 히스토리 남기기
    self.dW = None
    self.db = None
    

  def forward(self, X):
    self.original_x_shape = X.shape # 데이터의 원본형상 저장
    X = x.reshape(x.shape[0], -1) # 데이터를 1차원으로 변경
    self.X = X
    out = np.dot(self.X, self.W) + self.b

    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)

    # W 의 입력 신호 방향으로 흘러나가는 값은 입력값의 전치행렬 * 미분값
    self.dW = np.dot(self.X.T, dout)
    self.db = np.sum(dout, axis=0)
    
    dx = dx.reshape(*self.original_x_shape) # 입력 데이터 모양을 원래대로 변경 ( 텐서 대응 )

    return dx

In [42]:
! unzip common.zip

Archive:  common.zip
  inflating: functions.py            
  inflating: gradient.py             
  inflating: layers.py               
  inflating: multi_layer_net.py      
  inflating: multi_layer_net_extend.py  
  inflating: optimizer.py            
  inflating: trainer.py              
  inflating: util.py                 
 extracting: __init__.py             


In [43]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [54]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1 # 마지막 계층의 미분값 설정
        dout = self.lastLayer.backward(dout) # 마지막 계층에서의 미분값 전달 받기 (SoftMaxWithLoss에서 받음)
        
        layers = list(self.layers.values()) # 저장된 레이어를 불러와서 ( 여기서는 순차적인 레이어가 저장 되어 있음 )
        layers.reverse() # 뒤집음(뒤에서 부터 전달해야 하기 때문에 )
        
        # 뒤에서 부터 역전파
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [57]:
!pip download dataset

ERROR: Could not find a version that satisfies the requirement dataset.mnist (from versions: none)
ERROR: No matching distribution found for dataset.mnist


In [61]:
import sys, os
sys.path.append(os.pardir)
from tensorflow.keras import datasets
mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
network = TwoLayerNet(input_size=28*28, hidden_size=50, output_size=10)

X_batch = X_train[:3]
t_batch = y_train[:3]

grad_numerical = network.numerical_gradient(X_batch, t_batch)
grad_backpop   = network.gradient(X_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구하고 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backpop[key] - grad_numerical[key]))
    print(key + " : {}".format(diff))

W1 : 5.590728656240972e-07
b1 : 1.8890104108975486e-08
W2 : 6.547201369793953e-06
b2 : 8.565896433089526e-07
